# uwsgi

- 安装 `pip install uwsgi`
- 在项目中创建配置文件 
- `uwsgi.ini` 文件名可以随意指定，不过没有特殊的理由我们就使用这个
- 编辑配置文件

```python
[uwsgi]
 #使用nginx连接时使用
 #socket=127.0.0.1:8080
 #直接做web服务器使用
http=127.0.0.1:8080
 #项目目录
chdir=/home/python/dailyfresh/
 #项目中wsgi.py文件的目录，相对于项目目录
wsgi-file=dailyfresh/wsgi.py
 # 开启的工作进程数
processes=4
 # 每个进程的线程
threads=2
 # 主进程(用来监视工作进程，如果被杀死，再开启)
master=True
 # 保存主进程的进程id
pidfile=uwsgi.pid
 # 日志文件的保存位置
daemonize=uwsgi.log
 # 如果路径没有以/开头，则就是相对路径，都相对于chdir
virtualenv=/User/python/.virtualenvs/django_py3
```

- 启动uwsgi `uwsgi --ini uwsgi.ini`
- 测试uwsgi的开启情况
- 看进程 `ps aux|grep uwsgi`
- 使用浏览器访问服务器 `http://127.0.0.1:8080`
- 关闭uwsgi `uwsgi --stop uwsgi.pid`

# Nginx

- Nginx（发音同engine x）是俄罗斯人开发的一个 Web服务器，也可以用作反向代理，负载均衡和 HTTP缓存。 功能各种强大，性能各种高。
- 相关文档 
https://nginx.org/en/
http://tengine.taobao.org/book/
- nginx与uwsgi 
- - uwsgi处理动态请求能力高，但对于静态请求（如static文件，css,js文件等）处理能力差，此时就要结合nginx一起使用 

# Nginx安装

- 解压 `tar zxvf nginx-1.6.3.tar.gz`
- 执行命令进行安装

`./configure
make
sudo make install`

- 启动nginx `sudo /usr/local/nginx/sbin/nginx`

- 检测nginx的运行状态 
- - 看进程 
`ps aux|grep nginx`
- 通过浏览器访问127.0.0.1 

# nginx与uWSGI的使用

- 编辑nginx的配置文件 
`sudo gedit conf/nginx.conf`

- 在server节点下添加新的location项，指向uwsgi的ip与端口。 
- 使用#注释掉原有配置

```
location / {
    #将所有访问/的请求转到uwsgi下
    include uwsgi_params;
    #uwsgi的ip与端口
    uwsgi_pass uwsgi服务器的ip:端口;
}
```

- 修改uWSGI的配置文件

```
#使用nginx连接时使用
socket=127.0.0.1:8080
#直接做web服务器使用
#http=127.0.0.1:8080
```

- 开启uWSGI 
`uwsgi --ini uwsgi.ini`

- 开启nginx 
`sudo sbin/nginx`

# nginx与uWSGI的中使用静态文件

- 创建目录 
`sudo mkdir -p /var/www/dailyfresh/static/`
- 修改权限 
`sudo chmod 777 /var/www/dailyfresh/static/`
- **转移项目中的静态文件**
- 在settings.py中添加配置 
`STATIC_ROOT='/var/www/dailyfresh/static/'`
- 转移静态文件 
`python manage.py collectstatic`
- 配置nginx中静态文件的路径

```
location /static {
    #将所有访问/static的请求转到对应目录中
    alias /var/www/dailyfresh/static/;
}
```

- 重启(停止后再开启)uWSGI 

`uwsgi --stop uwsgi.pid`

`uwsgi --ini uwsgi.ini`

- 重启(停止后再开启)nginx 
`sudo nginx -s reload`

# nginx访问静态页面 
- 在调度nginx添加配置

```
location = / {
    proxy_pass http://静态文件所在的nginx的ip:port;
}
```

- 重启调度nginx 
`sudo nginx -s reload`

# nginx配置负载均衡 
- nginx作为调度服务器，也支持 应用(业务)服务器的负载均衡

### 步骤

- 准备多个uwsgi(同一电脑，不同ip；或者不同电脑都可以)
- 修改nginx的配置文件

- - 添加 upstream

```
upstream dailyfresh{
    server uwsgi1的ip:端口
    server uwsgi2的ip:端口
}
```
- - 修改 location /
```
location /{
    include uwsgi_params;
    # uwsgi_pass uwsgi1的ip:端口;
    uwsgi_pass dailyfresh;
}
```